In [1]:
#PyQt of 3 element filter with graph and sliders 

In [3]:
pip install pyqt5 opencv-python-headless numpy tifffile


Note: you may need to restart the kernel to use updated packages.


In [1]:
#Old Code Mouse data appears on the side
import os
import sys
import cv2
import numpy as np
import tifffile as tiff
from PyQt5.QtWidgets import (
    QApplication, QLabel, QWidget, QVBoxLayout,
    QPushButton, QHBoxLayout
)
from PyQt5.QtGui import QPixmap, QImage, QPainter, QColor, QPen
from PyQt5.QtCore import Qt

# === Set QT plugin path (needed for conda users) ===
os.environ["QT_QPA_PLATFORM_PLUGIN_PATH"] = os.path.join(
    os.environ["CONDA_PREFIX"], "lib", "qt", "plugins"
)

# === Image paths ===
img_r_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_Fe_merged.tiff"
img_g_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_Ca_merged.tiff"
img_b_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_S_merged.tiff"

# === Helper: normalize and dilate image ===
def normalize_and_dilate(img):
    img = np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0)
    norm = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    dilated = cv2.dilate(norm, np.ones((5, 5), np.uint8), iterations=3)
    return norm, dilated

# === Helper: blob detection ===
def detect_blobs(img_norm, img_orig, min_thresh):
    params = cv2.SimpleBlobDetector_Params()
    params.minThreshold = min_thresh
    params.maxThreshold = 255
    params.filterByArea = True
    params.minArea = 200
    params.maxArea = 50000
    params.filterByCircularity = False
    params.filterByConvexity = False
    params.filterByInertia = False
    params.filterByColor = True
    params.blobColor = 255
    detector = cv2.SimpleBlobDetector_create(params)

    keypoints = detector.detect(img_norm)
    blobs = []
    for kp in keypoints:
        x, y = int(kp.pt[0]), int(kp.pt[1])
        radius = int(kp.size / 2)
        mask = np.zeros(img_orig.shape, dtype=np.uint8)
        cv2.rectangle(mask, (x - radius, y - radius), (x + radius, y + radius), 255, thickness=-1)
        vals = img_orig[mask == 255]
        vals_dilated = img_norm[mask == 255]
        if vals.size > 0:
            blobs.append({
                'center': (x, y),
                'radius': radius,
                'size': kp.size,
                'max_intensity': vals.max(),
                'mean_intensity': vals.mean(),
                'mean_dilation': float(vals_dilated.mean())
            })
    return blobs

# === Load TIFF images and process ===
img_r = tiff.imread(img_r_path).astype(np.float32)
img_g = tiff.imread(img_g_path).astype(np.float32)
img_b = tiff.imread(img_b_path).astype(np.float32)

img_r_norm, img_r_dilated = normalize_and_dilate(img_r)
img_g_norm, img_g_dilated = normalize_and_dilate(img_g)
img_b_norm, img_b_dilated = normalize_and_dilate(img_b)

blobs_r = detect_blobs(img_r_dilated, img_r, min_thresh=100)
blobs_g = detect_blobs(img_g_dilated, img_g, min_thresh=100)
blobs_b = detect_blobs(img_b_dilated, img_b, min_thresh=100)

img_r_disp = cv2.normalize(img_r, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
img_g_disp = cv2.normalize(img_g, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
img_b_disp = cv2.normalize(img_b, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# === Merge channels to RGB ===
merged_rgb = cv2.merge([img_r_disp, img_g_disp, img_b_disp])  # [R, G, B]

# === Convert to QImage ===
height, width, channel = merged_rgb.shape
bytes_per_line = 3 * width
q_img = QImage(merged_rgb.data, width, height, bytes_per_line, QImage.Format_RGB888)

# === Draw bounding boxes on top ===
q_img_painted = QImage(q_img)  # Copy image for painting
painter = QPainter(q_img_painted)

pen_r = QPen(QColor("red"), 2)
pen_g = QPen(QColor("green"), 2)
pen_b = QPen(QColor("blue"), 2)

for blob in blobs_r:
    x, y = blob['center']
    r = blob['radius']
    painter.setPen(pen_r)
    painter.drawRect(x - r, y - r, 2 * r, 2 * r)

for blob in blobs_g:
    x, y = blob['center']
    r = blob['radius']
    painter.setPen(pen_g)
    painter.drawRect(x - r, y - r, 2 * r, 2 * r)

for blob in blobs_b:
    x, y = blob['center']
    r = blob['radius']
    painter.setPen(pen_b)
    painter.drawRect(x - r, y - r, 2 * r, 2 * r)

painter.end()

# === Hover-capable QLabel ===
class HoverLabel(QLabel):
    def __init__(self, coord_display_label, blobs_r, blobs_g, blobs_b):
        super().__init__()
        self.coord_display_label = coord_display_label
        self.setMouseTracking(True)
        self.blobs = {
            "Fe (Red)": blobs_r,
            "Ca (Green)": blobs_g,
            "S (Blue)": blobs_b
        }

    def mouseMoveEvent(self, event):
        x = event.pos().x()
        y = event.pos().y()
        label = f"X: {x}, Y: {y}"

        for element_name, blob_list in self.blobs.items():
            for blob in blob_list:
                bx, by = blob['center']
                r = blob['radius']
                if (bx - r) <= x <= (bx + r) and (by - r) <= y <= (by + r):
                    box_x, box_y = bx - r, by - r
                    box_size = 2 * r
                    label = (
                        f"<b>{element_name}</b><br>"
                        f"Mouse: ({x}, {y})<br>"
                        f"Center: ({bx}, {by})<br>"
                        f"Top-left: ({box_x}, {box_y})<br>"
                        f"Box size: {box_size} x {box_size} px<br>"
                        f"Box area: {box_size * box_size} px²<br>"
                        f"Max intensity: {blob['max_intensity']:.3f}<br>"
                        f"Mean intensity: {blob['mean_intensity']:.3f}<br>"
                        f"Mean dilation intensity: {blob['mean_dilation']:.1f}"
                    )
                    break
            else:
                continue
            break

        self.coord_display_label.setText(label)

# === GUI Setup ===
app = QApplication(sys.argv)
window = QWidget()
window.setWindowTitle("Merged TIFF Image with Blob Bounding Boxes")

main_layout = QHBoxLayout()

# === Coordinate label ===
coord_label = QLabel("Hover to see coordinates")
coord_label.setStyleSheet("font-size: 13px; padding: 4px;")
coord_label.setTextFormat(Qt.RichText)

# === Image area with hover tracking ===
image_label = HoverLabel(coord_label, blobs_r, blobs_g, blobs_b)
image_label.setPixmap(QPixmap.fromImage(q_img_painted))
main_layout.addWidget(image_label)

# === Control panel ===
control_panel = QWidget()
control_panel.setStyleSheet("background-color: white;")
control_layout = QVBoxLayout()

# Exit button
exit_btn = QPushButton("Exit")
exit_btn.setFixedWidth(100)
exit_btn.clicked.connect(window.close)
control_layout.addWidget(exit_btn)

# Coordinate display
control_layout.addWidget(coord_label)
control_layout.addStretch()

control_panel.setLayout(control_layout)
main_layout.addWidget(control_panel)

# Final layout
window.setLayout(main_layout)
window.show()
sys.exit(app.exec_())


qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.


SystemExit: 0

/home/codingcarlos/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:

img_r_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_Fe_merged.tiff"
img_g_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_Ca_merged.tiff"
img_b_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_S_merged.tiff"

img_r_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_175um_stitched/S.tiff"
img_g_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_175um_stitched/Al.tiff"
img_b_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_175um_stitched/Fe.tiff"

In [ ]:
#Below is 

In [1]:
import os
import sys
import cv2
import numpy as np
import tifffile as tiff
from PyQt5.QtWidgets import (
    QApplication, QLabel, QWidget, QVBoxLayout, QHBoxLayout,
    QPushButton, QGraphicsView, QGraphicsScene, QGraphicsPixmapItem,
    QCheckBox, QSlider, QFileDialog
)
from PyQt5.QtGui import QPixmap, QImage, QPainter, QColor, QPen
from PyQt5.QtCore import Qt
from collections import Counter

# Store paths selected by user
img_paths = [None, None, None]  # [R, G, B]
file_names = [None, None, None]
element_colors = ["red", "green", "blue"]

app = QApplication(sys.argv)
window = QWidget()
window.setWindowTitle("TIFF Viewer — Choose Files First")

main_layout = QVBoxLayout()
file_select_layout = QHBoxLayout()
file_select_layout.setSpacing(20)  # Horizontal spacing between columns

labels = []
buttons = []

def handle_file_selection(index):
    path, _ = QFileDialog.getOpenFileName(window, f"Select Element {index+1} TIFF", "", "TIFF Files (*.tiff *.tif)")
    if path:
        img_paths[index] = path
        file_names[index] = os.path.basename(path)
        labels[index].setText(f"Element {index+1}: {file_names[index]}")
    if all(img_paths):
        init_gui()

for i in range(3):
    vbox = QVBoxLayout()
    lbl = QLabel(f"Element {i+1}: Not selected")
    btn = QPushButton(f"Choose Element {i+1}")
    btn.clicked.connect(lambda _, idx=i: handle_file_selection(idx))
    vbox.addWidget(lbl)
    vbox.addWidget(btn)
    labels.append(lbl)
    buttons.append(btn)
    file_select_layout.addLayout(vbox)


center_layout = QVBoxLayout()
center_layout.setAlignment(Qt.AlignCenter)  # Center on screen
center_layout.addLayout(file_select_layout)
main_layout.addLayout(center_layout)

# === Defer the rest of the UI until files are selected ===
def init_gui():
    for btn in buttons:
        btn.setEnabled(False)

    # Load and convert
    img_r, img_g, img_b = [tiff.imread(p).astype(np.float32) for p in img_paths]
    
    # Resize to majority shape
    shapes = [img_r.shape, img_g.shape, img_b.shape]
    shape_counts = Counter(shapes)
    target_shape = shape_counts.most_common(1)[0][0]
    print(f"Target (majority) shape: {target_shape}")
    
    def resize_if_needed(img, name):
        if img.shape != target_shape:
            print(f"Resizing {name} from {img.shape} → {target_shape}")
            return cv2.resize(img, (target_shape[1], target_shape[0]), interpolation=cv2.INTER_AREA)
        return img
    
    img_r = resize_if_needed(img_r, file_names[0])
    img_g = resize_if_needed(img_g, file_names[1])
    img_b = resize_if_needed(img_b, file_names[2])

    def normalize_and_dilate(img):
        img = np.nan_to_num(img)
        norm = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        dilated = cv2.dilate(norm, np.ones((5, 5), np.uint8), iterations=3)
        return norm, dilated

    def detect_blobs(img_norm, img_orig, min_thresh, min_area, color, file_name):
        params = cv2.SimpleBlobDetector_Params()
        params.minThreshold = min_thresh
        params.maxThreshold = 255
        params.filterByArea = True
        params.minArea = min_area
        params.maxArea = 50000
        params.filterByColor = True
        params.blobColor = 255
        params.minRepeatability = 1
        detector = cv2.SimpleBlobDetector_create(params)
        keypoints = detector.detect(img_norm)
        blobs = []
        for kp in keypoints:
            x, y = int(kp.pt[0]), int(kp.pt[1])
            radius = int(kp.size / 2)
            box_size = 2 * radius
            box_x, box_y = x - radius, y - radius
            mask = np.zeros(img_orig.shape, dtype=np.uint8)
            cv2.rectangle(mask, (box_x, box_y), (x + radius, y + radius), 255, thickness=-1)
            vals = img_orig[mask == 255]
            vals_dilated = img_norm[mask == 255]
            if vals.size > 0:
                blobs.append({
                    'center': (x, y),
                    'radius': radius,
                    'color': color,
                    'file': file_name,
                    'max_intensity': vals.max(),
                    'mean_intensity': vals.mean(),
                    'mean_dilation': float(vals_dilated.mean()),
                    'box_x': box_x,
                    'box_y': box_y,
                    'box_size': box_size
                })
        return blobs

    norm_dilated = [normalize_and_dilate(im) for im in [img_r, img_g, img_b]]
    normalized = [nd[0] for nd in norm_dilated]
    dilated = [nd[1] for nd in norm_dilated]

    thresholds_range = list(range(0, 256, 10))
    area_range = list(range(100, 501, 50))
    precomputed_blobs = {
        color: {
            (t, a): detect_blobs(dilated[i], [img_r, img_g, img_b][i], t, a, color, file_names[i])
            for t in thresholds_range
            for a in area_range
        }
        for i, color in enumerate(element_colors)
    }

    thresholds = {color: 100 for color in element_colors}
    area_thresholds = {color: 200 for color in element_colors}

    def get_current_blobs():
        blobs = []
        for color in element_colors:
            thresh = thresholds[color]
            area = area_thresholds[color]
            
            # Snap area to nearest available area_range value (optional, if mismatch risk exists)
            available_areas = list(range(100, 501, 50))
            snapped_area = min(available_areas, key=lambda a: abs(a - area))
            
            key = (thresh, snapped_area)
            blobs_for_color = precomputed_blobs[color].get(key, [])
            
            filtered = blobs_for_color  # no need to filter again, already done

            blobs.extend(filtered)
        return blobs
    
    merged_rgb = cv2.merge([
        cv2.normalize(img_r, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8),
        cv2.normalize(img_g, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8),
        cv2.normalize(img_b, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    ])

    hover_label = QLabel()
    hover_label.setWindowFlags(Qt.ToolTip)
    hover_label.hide()

    x_label = QLabel("X: 0")
    y_label = QLabel("Y: 0")

    scene = QGraphicsScene()
    q_img = QImage(merged_rgb.data, merged_rgb.shape[1], merged_rgb.shape[0], merged_rgb.shape[1] * 3, QImage.Format_RGB888)
    pixmap_item = QGraphicsPixmapItem(QPixmap.fromImage(q_img))
    scene.addItem(pixmap_item)

    class ZoomableGraphicsView(QGraphicsView):
        def __init__(self, scene, hover_label, x_label, y_label):
            super().__init__(scene)
            self.setRenderHints(QPainter.Antialiasing | QPainter.SmoothPixmapTransform)
            self.setTransformationAnchor(QGraphicsView.AnchorUnderMouse)
            self.setMouseTracking(True)
            self.setDragMode(QGraphicsView.NoDrag)
            self._drag_active = False
            self.hover_label = hover_label
            self.blobs = []
            self.visible_colors = set(element_colors)
            self.x_label = x_label
            self.y_label = y_label

        def wheelEvent(self, event):
            cursor_pos = event.pos()
            scene_pos = self.mapToScene(cursor_pos)
            zoom_factor = 1.25 if event.angleDelta().y() > 0 else 0.8
            self.scale(zoom_factor, zoom_factor)
            mouse_centered = self.mapFromScene(scene_pos)
            delta = cursor_pos - mouse_centered
            self.horizontalScrollBar().setValue(self.horizontalScrollBar().value() - delta.x())
            self.verticalScrollBar().setValue(self.verticalScrollBar().value() - delta.y())

        def mousePressEvent(self, event):
            if event.button() == Qt.LeftButton:
                self.setDragMode(QGraphicsView.ScrollHandDrag)
                self._drag_active = True
            super().mousePressEvent(event)

        def mouseReleaseEvent(self, event):
            if event.button() == Qt.LeftButton:
                self.setDragMode(QGraphicsView.NoDrag)
                self._drag_active = False
            super().mouseReleaseEvent(event)

        def mouseMoveEvent(self, event):
            if self._drag_active:
                super().mouseMoveEvent(event)
                return
            pos = self.mapToScene(event.pos())
            x, y = int(pos.x()), int(pos.y())
            self.x_label.setText(f"X: {x}")
            self.y_label.setText(f"Y: {y}")
            for blob in self.blobs:
                if blob['color'] not in self.visible_colors:
                    continue
                cx, cy = blob['center']
                r = blob['radius']
                if abs(x - cx) <= r and abs(y - cy) <= r:
                    html = (
                        f"<b>{blob['color'].capitalize()} Element</b><br>"
                        f"<i>{blob['file']}</i><br>"
                        f"Center: ({cx}, {cy})<br>"
                        f"Top-left: ({blob['box_x']}, {blob['box_y']})<br>"
                        f"Box size: {blob['box_size']} x {blob['box_size']} px<br>"
                        f"Box area: {blob['box_size'] * blob['box_size']} px²<br>"
                        f"Max intensity: {blob['max_intensity']:.3f}<br>"
                        f"Mean intensity: {blob['mean_intensity']:.3f}<br>"
                        f"Mean dilation intensity: {blob['mean_dilation']:.1f}"
                    )
                    self.hover_label.setText(html)
                    self.hover_label.adjustSize()
                    self.hover_label.move(event.x() + 15, event.y() - 30)
                    self.hover_label.setStyleSheet(
                        f"background-color: {blob['color']}; color: white; border: 1px solid black; padding: 4px;"
                    )
                    self.hover_label.show()
                    return
            self.hover_label.hide()

        def update_blobs(self, blobs, visible_colors):
            self.blobs = blobs
            self.visible_colors = visible_colors

    graphics_view = ZoomableGraphicsView(scene, hover_label, x_label, y_label)

    def redraw_boxes(blobs, selected_colors):
        updated_img = QImage(q_img)
        painter = QPainter(updated_img)
        for blob in blobs:
            if blob['color'] in selected_colors:
                cx, cy, r = *blob['center'], blob['radius']
                painter.setPen(QPen(QColor(blob['color']), 2))
                painter.drawRect(cx - r, cy - r, 2 * r, 2 * r)
        painter.end()
        pixmap_item.setPixmap(QPixmap.fromImage(updated_img))

    checkboxes = {}
    selected_colors = set(element_colors)

    def update_boxes():
        nonlocal selected_colors
        selected_colors = {c for c, cb in checkboxes.items() if cb.isChecked()}
        blobs = [b for b in get_current_blobs() if b['color'] in selected_colors]
        graphics_view.update_blobs(blobs, selected_colors)
        redraw_boxes(blobs, selected_colors)
        hover_label.hide()

    legend_layout = QVBoxLayout()
    legend_label = QLabel("Legend")
    legend_label.setStyleSheet("font-weight: bold; font-size: 12pt;")
    legend_layout.addWidget(legend_label)
    for i, color in enumerate(element_colors):
        cb = QCheckBox(file_names[i])
        cb.setChecked(True)
        cb.setStyleSheet(f"color: {color}")
        cb.stateChanged.connect(update_boxes)
        checkboxes[color] = cb
        legend_layout.addWidget(cb)
    legend_layout.addStretch()

    sliders = {}
    slider_labels = {}
    slider_layout = QHBoxLayout()

    def on_slider_change(value, color):
        snapped = round(value / 10) * 10
        snapped = max(0, min(250, snapped))
        if thresholds[color] != snapped:
            thresholds[color] = snapped
            sliders[color].blockSignals(True)
            sliders[color].setValue(snapped)
            sliders[color].blockSignals(False)
            slider_labels[color].setText(f"{checkboxes[color].text()}_threshold: {snapped}")
            update_boxes()
        
    for color in element_colors:
        i = element_colors.index(color)
        vbox = QVBoxLayout()
        label = QLabel(f"{file_names[i]}_threshold: {thresholds[color]}")
        slider = QSlider(Qt.Horizontal)
        slider.setMinimum(0)
        slider.setMaximum(255)
        slider.setTickInterval(10)
        slider.setValue(thresholds[color])
        slider.setTickPosition(QSlider.TicksBelow)
        slider.valueChanged.connect(lambda val, c=color: on_slider_change(val, c))
        sliders[color] = slider
        slider_labels[color] = label
        vbox.addWidget(label)
        vbox.addWidget(slider)
        slider_layout.addLayout(vbox)

    area_sliders = {}
    area_slider_labels = {}
    
    def on_area_slider_change(value, color):
        valid_areas = list(range(100, 501, 50))  # [100, 150, ..., 500]
        snapped = min(valid_areas, key=lambda a: abs(a - value))
        if area_thresholds[color] != snapped:
            area_thresholds[color] = snapped
            area_sliders[color].blockSignals(True)
            area_sliders[color].setValue(snapped)
            area_sliders[color].blockSignals(False)
            area_slider_labels[color].setText(f"{checkboxes[color].text()}_min_area: {snapped}")
            update_boxes()


    area_slider_layout = QHBoxLayout()
    
    for color in element_colors:
        i = element_colors.index(color)
        vbox = QVBoxLayout()
        label = QLabel(f"{file_names[i]}_min_area: {area_thresholds[color]}")
        slider = QSlider(Qt.Horizontal)
        slider.setMinimum(100)
        slider.setMaximum(500)
        slider.setTickInterval(50)
        slider.setValue(area_thresholds[color])
        slider.setTickPosition(QSlider.TicksBelow)
        slider.valueChanged.connect(lambda val, c=color: on_area_slider_change(val, c))
        area_sliders[color] = slider
        area_slider_labels[color] = label
        vbox.addWidget(label)
        vbox.addWidget(slider)
        area_slider_layout.addLayout(vbox)

    exit_btn = QPushButton("Exit")
    exit_btn.clicked.connect(window.close)
    reset_btn = QPushButton("Reset View")
    reset_btn.clicked.connect(lambda: graphics_view.resetTransform())

    controls = QVBoxLayout()
    controls.addWidget(exit_btn)
    controls.addWidget(reset_btn)
    controls.addLayout(legend_layout)
    controls.addLayout(slider_layout)
    controls.addLayout(area_slider_layout)
    controls.addWidget(x_label)
    controls.addWidget(y_label)

    layout = QHBoxLayout()
    layout.addWidget(graphics_view)
    side_panel = QWidget()
    side_panel.setLayout(controls)
    layout.addWidget(side_panel)

    main_layout.addLayout(layout)
    hover_label.setParent(window)

    blobs = get_current_blobs()
    graphics_view.update_blobs(blobs, selected_colors)
    redraw_boxes(blobs, selected_colors)

    window.resize(1200, 800)

window.setLayout(main_layout)
window.show()
sys.exit(app.exec_())


qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.
qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1

Target (majority) shape: (1039, 1039)


SystemExit: 0

/home/codingcarlos/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# min area 200


In [ ]:
#Added checklist 

In [1]:
import os
import sys
import cv2
import numpy as np
import tifffile as tiff
from PyQt5.QtWidgets import (
    QApplication, QLabel, QWidget, QVBoxLayout, QHBoxLayout,
    QPushButton, QGraphicsView, QGraphicsScene, QGraphicsPixmapItem,
    QCheckBox, QSlider, QFileDialog, QListWidget, QListWidgetItem, QMessageBox
)
from PyQt5.QtGui import QPixmap, QImage, QPainter, QColor, QPen
from PyQt5.QtCore import Qt
from collections import Counter

# Store paths selected by user
img_paths = [None, None, None]  # [R, G, B]
file_names = [None, None, None]
element_colors = ["red", "green", "blue"]

graphics_view = None
controls_widget = None

def on_dir_selected():
    directory = QFileDialog.getExistingDirectory(window, "Select Directory")
    if not directory:
        return

    all_files = sorted(os.listdir(directory))  # Alphabetical (case-sensitive)
    file_list_widget.clear()
    file_paths.clear()

    for fname in all_files:
        full_path = os.path.join(directory, fname)
        if os.path.isfile(full_path):
            item = QListWidgetItem(f"{fname} ({os.path.splitext(fname)[1][1:].upper()})")
            item.setCheckState(Qt.Unchecked)
            file_list_widget.addItem(item)
            file_paths.append(full_path)

# Keep these global to track order of selection
selected_files_order = []  # store indices of items checked, in order

def update_selection():
    global selected_files_order

    # Check which items are checked currently
    checked_indices = [i for i in range(file_list_widget.count()) if file_list_widget.item(i).checkState() == Qt.Checked]

    # Find newly checked or unchecked items by comparing to stored order
    # Remove unchecked items from order
    selected_files_order = [i for i in selected_files_order if i in checked_indices]

    # Add newly checked items at the end
    for i in checked_indices:
        if i not in selected_files_order:
            if len(selected_files_order) < 3:
                selected_files_order.append(i)
            else:
                # Too many selected, uncheck this item immediately
                file_list_widget.item(i).setCheckState(Qt.Unchecked)

    # If less than 3 selected, clear img_paths and file_names partially
    for idx in range(3):
        if idx < len(selected_files_order):
            path = file_paths[selected_files_order[idx]]
            img_paths[idx] = path
            file_names[idx] = os.path.basename(path)
        else:
            img_paths[idx] = None
            file_names[idx] = None
 
def on_confirm_clicked():
    if len(selected_files_order) != 3:
        QMessageBox.warning(window, "Invalid Selection", "Please select exactly 3 items.")
        return

    QMessageBox.information(window, "Loading", "Now loading graph...")
    init_gui()


app = QApplication(sys.argv)
window = QWidget()
window.setWindowTitle("Select 3 TIFF Files for Elements")
window.resize(600, 500)

main_layout = QVBoxLayout()

# Directory selector
dir_button = QPushButton("Choose Directory")
dir_button.clicked.connect(on_dir_selected)
main_layout.addWidget(dir_button)

# List of files in the selected directory
file_list_widget = QListWidget()
file_list_widget.setSelectionMode(QListWidget.NoSelection)
file_list_widget.itemChanged.connect(update_selection)
# Horizontal layout: file list on the left, confirm button on the right
file_confirm_layout = QHBoxLayout()
file_confirm_layout.addWidget(file_list_widget)

# Right side (confirm button)
right_panel = QVBoxLayout()
confirm_button = QPushButton("Confirm Selection")
confirm_button.clicked.connect(on_confirm_clicked)
right_panel.addWidget(confirm_button)
right_panel.addStretch()
file_confirm_layout.addLayout(right_panel)

main_layout.addLayout(file_confirm_layout)

# Track full file paths
file_paths = []

# === Defer the rest of the UI until files are selected ===
def init_gui():       
    # for btn in buttons:
    #     btn.setEnabled(False)

    # Load and convert
    global graphics_view, controls_widget

    if graphics_view is not None:
        main_layout.removeWidget(graphics_view)
        graphics_view.setParent(None)
        graphics_view.deleteLater()
        graphics_view = None

    if controls_widget is not None:
        main_layout.removeWidget(controls_widget)
        controls_widget.setParent(None)
        controls_widget.deleteLater()
        controls_widget = None
        
    img_r, img_g, img_b = [tiff.imread(p).astype(np.float32) for p in img_paths]
    
    # Resize to majority shape
    shapes = [img_r.shape, img_g.shape, img_b.shape]
    shape_counts = Counter(shapes)
    target_shape = shape_counts.most_common(1)[0][0]
    print(f"Target (majority) shape: {target_shape}")
    
    def resize_if_needed(img, name):
        if img.shape != target_shape:
            print(f"Resizing {name} from {img.shape} → {target_shape}")
            return cv2.resize(img, (target_shape[1], target_shape[0]), interpolation=cv2.INTER_AREA)
        return img
    
    img_r = resize_if_needed(img_r, file_names[0])
    img_g = resize_if_needed(img_g, file_names[1])
    img_b = resize_if_needed(img_b, file_names[2])

    def normalize_and_dilate(img):
        img = np.nan_to_num(img)
        norm = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        dilated = cv2.dilate(norm, np.ones((5, 5), np.uint8), iterations=3)
        return norm, dilated

    def detect_blobs(img_norm, img_orig, min_thresh, min_area, color, file_name):
        params = cv2.SimpleBlobDetector_Params()
        params.minThreshold = min_thresh
        params.maxThreshold = 255
        params.filterByArea = True
        params.minArea = min_area
        params.maxArea = 50000
        params.filterByColor = True
        params.blobColor = 255
        params.minRepeatability = 1
        detector = cv2.SimpleBlobDetector_create(params)
        keypoints = detector.detect(img_norm)
        blobs = []
        for kp in keypoints:
            x, y = int(kp.pt[0]), int(kp.pt[1])
            radius = int(kp.size / 2)
            box_size = 2 * radius
            box_x, box_y = x - radius, y - radius
            mask = np.zeros(img_orig.shape, dtype=np.uint8)
            cv2.rectangle(mask, (box_x, box_y), (x + radius, y + radius), 255, thickness=-1)
            vals = img_orig[mask == 255]
            vals_dilated = img_norm[mask == 255]
            if vals.size > 0:
                blobs.append({
                    'center': (x, y),
                    'radius': radius,
                    'color': color,
                    'file': file_name,
                    'max_intensity': vals.max(),
                    'mean_intensity': vals.mean(),
                    'mean_dilation': float(vals_dilated.mean()),
                    'box_x': box_x,
                    'box_y': box_y,
                    'box_size': box_size
                })
        return blobs

    norm_dilated = [normalize_and_dilate(im) for im in [img_r, img_g, img_b]]
    normalized = [nd[0] for nd in norm_dilated]
    dilated = [nd[1] for nd in norm_dilated]

    thresholds_range = list(range(0, 256, 10))
    area_range = list(range(10, 501, 10))
    precomputed_blobs = {
        color: {
            (t, a): detect_blobs(dilated[i], [img_r, img_g, img_b][i], t, a, color, file_names[i])
            for t in thresholds_range
            for a in area_range
        }
        for i, color in enumerate(element_colors)
    }

    thresholds = {color: 100 for color in element_colors}
    area_thresholds = {color: 200 for color in element_colors}

    def get_current_blobs():
        blobs = []
        for color in element_colors:
            thresh = thresholds[color]
            area = area_thresholds[color]
            
            # Snap area to nearest available area_range value (optional, if mismatch risk exists)
            available_areas = list(range(10, 501, 10))
            snapped_area = min(available_areas, key=lambda a: abs(a - area))
            
            key = (thresh, snapped_area)
            blobs_for_color = precomputed_blobs[color].get(key, [])
            
            filtered = blobs_for_color  # no need to filter again, already done

            blobs.extend(filtered)
        return blobs
    
    merged_rgb = cv2.merge([
        cv2.normalize(img_r, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8),
        cv2.normalize(img_g, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8),
        cv2.normalize(img_b, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    ])

    hover_label = QLabel()
    hover_label.setWindowFlags(Qt.ToolTip)
    hover_label.hide()

    x_label = QLabel("X: 0")
    y_label = QLabel("Y: 0")

    scene = QGraphicsScene()
    q_img = QImage(merged_rgb.data, merged_rgb.shape[1], merged_rgb.shape[0], merged_rgb.shape[1] * 3, QImage.Format_RGB888)
    pixmap_item = QGraphicsPixmapItem(QPixmap.fromImage(q_img))
    scene.addItem(pixmap_item)

    class ZoomableGraphicsView(QGraphicsView):
        def __init__(self, scene, hover_label, x_label, y_label):
            super().__init__(scene)
            self.setRenderHints(QPainter.Antialiasing | QPainter.SmoothPixmapTransform)
            self.setTransformationAnchor(QGraphicsView.AnchorUnderMouse)
            self.setMouseTracking(True)
            self.setDragMode(QGraphicsView.NoDrag)
            self._drag_active = False
            self.hover_label = hover_label
            self.blobs = []
            self.visible_colors = set(element_colors)
            self.x_label = x_label
            self.y_label = y_label

        def wheelEvent(self, event):
            cursor_pos = event.pos()
            scene_pos = self.mapToScene(cursor_pos)
            zoom_factor = 1.25 if event.angleDelta().y() > 0 else 0.8
            self.scale(zoom_factor, zoom_factor)
            mouse_centered = self.mapFromScene(scene_pos)
            delta = cursor_pos - mouse_centered
            self.horizontalScrollBar().setValue(self.horizontalScrollBar().value() - delta.x())
            self.verticalScrollBar().setValue(self.verticalScrollBar().value() - delta.y())

        def mousePressEvent(self, event):
            if event.button() == Qt.LeftButton:
                self.setDragMode(QGraphicsView.ScrollHandDrag)
                self._drag_active = True
            super().mousePressEvent(event)

        def mouseReleaseEvent(self, event):
            if event.button() == Qt.LeftButton:
                self.setDragMode(QGraphicsView.NoDrag)
                self._drag_active = False
            super().mouseReleaseEvent(event)

        def mouseMoveEvent(self, event):
            if self._drag_active:
                super().mouseMoveEvent(event)
                return
            pos = self.mapToScene(event.pos())
            x, y = int(pos.x()), int(pos.y())
            self.x_label.setText(f"X: {x}")
            self.y_label.setText(f"Y: {y}")
            for blob in self.blobs:
                if blob['color'] not in self.visible_colors:
                    continue
                cx, cy = blob['center']
                r = blob['radius']
                if abs(x - cx) <= r and abs(y - cy) <= r:
                    html = (
                        f"<b>{blob['color'].capitalize()} Element</b><br>"
                        f"<i>{blob['file']}</i><br>"
                        f"Center: ({cx}, {cy})<br>"
                        f"Top-left: ({blob['box_x']}, {blob['box_y']})<br>"
                        f"Box size: {blob['box_size']} x {blob['box_size']} px<br>"
                        f"Box area: {blob['box_size'] * blob['box_size']} px²<br>"
                        f"Max intensity: {blob['max_intensity']:.3f}<br>"
                        f"Mean intensity: {blob['mean_intensity']:.3f}<br>"
                        f"Mean dilation intensity: {blob['mean_dilation']:.1f}"
                    )
                    self.hover_label.setText(html)
                    self.hover_label.adjustSize()
                    self.hover_label.move(event.x() + 15, event.y() - 30)
                    self.hover_label.setStyleSheet(
                        f"background-color: {blob['color']}; color: white; border: 1px solid black; padding: 4px;"
                    )
                    self.hover_label.show()
                    return
            self.hover_label.hide()

        def update_blobs(self, blobs, visible_colors):
            self.blobs = blobs
            self.visible_colors = visible_colors

    graphics_view = ZoomableGraphicsView(scene, hover_label, x_label, y_label)
    main_layout.addWidget(graphics_view)
    
    def redraw_boxes(blobs, selected_colors):
        updated_img = QImage(q_img)
        painter = QPainter(updated_img)
        for blob in blobs:
            if blob['color'] in selected_colors:
                cx, cy, r = *blob['center'], blob['radius']
                painter.setPen(QPen(QColor(blob['color']), 2))
                painter.drawRect(cx - r, cy - r, 2 * r, 2 * r)
        painter.end()
        pixmap_item.setPixmap(QPixmap.fromImage(updated_img))

    checkboxes = {}
    selected_colors = set(element_colors)

    def update_boxes():
        nonlocal selected_colors
        selected_colors = {c for c, cb in checkboxes.items() if cb.isChecked()}
        blobs = [b for b in get_current_blobs() if b['color'] in selected_colors]
        graphics_view.update_blobs(blobs, selected_colors)
        redraw_boxes(blobs, selected_colors)
        hover_label.hide()

    legend_layout = QVBoxLayout()
    legend_label = QLabel("Legend")
    legend_label.setStyleSheet("font-weight: bold; font-size: 12pt;")
    legend_layout.addWidget(legend_label)
    for i, color in enumerate(element_colors):
        cb = QCheckBox(file_names[i])
        cb.setChecked(True)
        cb.setStyleSheet(f"color: {color}")
        cb.stateChanged.connect(update_boxes)
        checkboxes[color] = cb
        legend_layout.addWidget(cb)
    legend_layout.addStretch()

    sliders = {}
    slider_labels = {}
    slider_layout = QHBoxLayout()

    def on_slider_change(value, color):
        snapped = round(value / 10) * 10
        snapped = max(0, min(250, snapped))
        if thresholds[color] != snapped:
            thresholds[color] = snapped
            sliders[color].blockSignals(True)
            sliders[color].setValue(snapped)
            sliders[color].blockSignals(False)
            slider_labels[color].setText(f"{checkboxes[color].text()}_threshold: {snapped}")
            update_boxes()
        
    for color in element_colors:
        i = element_colors.index(color)
        vbox = QVBoxLayout()
        label = QLabel(f"{file_names[i]}_threshold: {thresholds[color]}")
        slider = QSlider(Qt.Horizontal)
        slider.setMinimum(0)
        slider.setMaximum(255)
        slider.setTickInterval(10)
        slider.setValue(thresholds[color])
        slider.setTickPosition(QSlider.TicksBelow)
        slider.valueChanged.connect(lambda val, c=color: on_slider_change(val, c))
        sliders[color] = slider
        slider_labels[color] = label
        vbox.addWidget(label)
        vbox.addWidget(slider)
        slider_layout.addLayout(vbox)

    area_sliders = {}
    area_slider_labels = {}
    
    def on_area_slider_change(value, color):
        valid_areas = list(range(10, 501, 10))  # [100, 150, ..., 500]
        snapped = min(valid_areas, key=lambda a: abs(a - value))
        if area_thresholds[color] != snapped:
            area_thresholds[color] = snapped
            area_sliders[color].blockSignals(True)
            area_sliders[color].setValue(snapped)
            area_sliders[color].blockSignals(False)
            area_slider_labels[color].setText(f"{checkboxes[color].text()}_min_area: {snapped}")
            update_boxes()


    area_slider_layout = QHBoxLayout()
    
    for color in element_colors:
        i = element_colors.index(color)
        vbox = QVBoxLayout()
        label = QLabel(f"{file_names[i]}_min_area: {area_thresholds[color]}")
        slider = QSlider(Qt.Horizontal)
        slider.setMinimum(10)
        slider.setMaximum(500)
        slider.setTickInterval(10)
        slider.setValue(area_thresholds[color])
        slider.setTickPosition(QSlider.TicksBelow)
        slider.valueChanged.connect(lambda val, c=color: on_area_slider_change(val, c))
        area_sliders[color] = slider
        area_slider_labels[color] = label
        vbox.addWidget(label)
        vbox.addWidget(slider)
        area_slider_layout.addLayout(vbox)

    exit_btn = QPushButton("Exit")
    exit_btn.clicked.connect(window.close)
    reset_btn = QPushButton("Reset View")
    reset_btn.clicked.connect(lambda: graphics_view.resetTransform())

    controls = QVBoxLayout()
    controls.addWidget(exit_btn)
    controls.addWidget(reset_btn)
    controls.addLayout(legend_layout)
    controls.addLayout(slider_layout)
    controls.addLayout(area_slider_layout)
    controls.addWidget(x_label)
    controls.addWidget(y_label)

    layout = QHBoxLayout()
    layout.addWidget(graphics_view)
    side_panel = QWidget()
    side_panel.setLayout(controls)
    controls_widget = side_panel
    layout.addWidget(side_panel)

    main_layout.addLayout(layout)
    hover_label.setParent(window)

    blobs = get_current_blobs()
    graphics_view.update_blobs(blobs, selected_colors)
    redraw_boxes(blobs, selected_colors)

    window.resize(1200, 800)

window.setLayout(main_layout)
window.show()
sys.exit(app.exec_())


qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.
qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1

Target (majority) shape: (1039, 1039)


SystemExit: 0

/home/codingcarlos/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
works great but the min threshold does get some tiny stuff

work on origin manually offset

and pixels per micron for real location 